<a href="https://colab.research.google.com/github/Rohitkumar-88/ML_Projects/blob/Colab_Notebooks/NER_Prediction_with_LSTM_and_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download the ner_dataset.csv file from this site (https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus?select=ner_dataset.csv)

In [ ]:
!wget https://raw.githubusercontent.com/amm-ik/ml-datasets/main/module-5/ner_dataset.csv

--2023-07-11 03:31:19--  https://raw.githubusercontent.com/amm-ik/ml-datasets/main/module-5/ner_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15208151 (15M) [text/plain]
Saving to: ‘ner_dataset.csv’

ner_dataset.csv     100%[===================>]  14.50M  --.-KB/s    in 0.05s   

2023-07-11 03:31:22 (300 MB/s) - ‘ner_dataset.csv’ saved [15208151/15208151]



Import the libraries we will need.

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from itertools import chain
import tensorflow

Let's look at the structure of the data

In [ ]:
data = pd.read_csv('ner_dataset.csv', encoding= 'unicode_escape')
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


We next need to create a mapping between tokens, tags, and ids. Each token should map to a unique id, and each tag should map to a unique class.

In [ ]:
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

print("Token for id 0 :", idx2token[0])
print("Tag for id 0: ", idx2tag[0])

Token for id 0 : four-day
Tag for id 0:  B-per


In [ ]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data.head(10)

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,Thousands,NNS,O,3766,2
1,NaN,of,IN,O,15128,2
2,NaN,demonstrators,NNS,O,4472,2
3,NaN,have,VBP,O,13943,2
4,NaN,marched,VBN,O,25841,2
5,NaN,through,IN,O,14753,2
6,NaN,London,NNP,B-geo,8278,6
7,NaN,to,TO,O,32297,2
8,NaN,protest,VB,O,14431,2
9,NaN,the,DT,O,35142,2


Now you might have noticed that each sentece is split into multiple rows. We need to transform this data into sequences of words and tags.

In [ ]:
# Fill na
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))
# Visualise data
data_group.head(10)

<ipython-input-6-02e79fa7aa5c>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[3766, 15128, 4472, 13943, 25841, 14753, 8278,...","[2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 6, 2, 2, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[2301, 2702, 27099, 28428, 14984, 32297, 7689,...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[3682, 16698, 473, 6832, 23992, 21987, 26824, ...","[2, 2, 16, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 1,..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[16112, 13400, 10371, 4283, 22790, 33063, 7619...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[14840, 20119, 26950, 14390, 884, 8454, 21437,...","[6, 2, 2, 0, 10, 2, 16, 2, 6, 2, 3, 2, 3, 2, 2..."
5,Sentence: 10001,"[Mr., Egeland, said, the, latest, figures, sho...","[NNP, NNP, VBD, DT, JJS, NNS, VBP, CD, CD, NNS...","[B-per, I-per, O, O, O, O, O, O, O, O, O, O, O...","[3369, 884, 8454, 35142, 27581, 6071, 21256, 2...","[0, 10, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,..."
6,Sentence: 10002,"[He, said, last, week, 's, tsunami, and, the, ...","[PRP, VBD, JJ, NN, POS, NN, CC, DT, JJ, NN, NN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[14670, 8454, 25056, 11728, 6857, 28598, 32957...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
7,Sentence: 10003,"[Some, 1,27,000, people, are, known, dead, .]","[DT, CD, NNS, VBP, VBN, JJ, .]","[O, O, O, O, O, O, O]","[26845, 23479, 29034, 15753, 30330, 13399, 32172]","[2, 2, 2, 2, 2, 2, 2]"
8,Sentence: 10004,"[Aid, is, being, rushed, to, the, region, ,, b...","[NNP, VBZ, VBG, VBN, TO, DT, NN, ,, CC, DT, NN...","[O, O, O, O, O, O, O, O, O, O, B-geo, O, O, O,...","[29818, 17852, 1690, 12072, 32297, 35142, 3435...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, ..."
9,Sentence: 10005,"[Lebanese, politicians, are, condemning, Frida...","[JJ, NNS, VBP, VBG, NNP, POS, NN, NN, IN, DT, ...","[B-gpe, O, O, O, B-tim, O, O, O, O, O, O, O, O...","[6624, 1724, 15753, 23233, 11901, 6857, 27688,...","[3, 2, 2, 2, 16, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2,..."


Next we split the data into training and testing

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils import pad_sequences
from tensorflow.keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


Before creating our model, we need to find some pre-trained embeddings to help us with the task...for example, you can find GloVe embeddings here https://nlp.stanford.edu/projects/glove/

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2023-07-11 03:35:43--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-07-11 03:35:43--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.13MB/s    in 2m 40s  

2023-07-11 03:38:24 (5.14 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!pip install patool
import patoolib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
patoolib.extract_archive("glove.6B.zip", outdir="./")

patool: Extracting glove.6B.zip ...
patool: running /usr/bin/7z x -o./ -- glove.6B.zip
patool: ... glove.6B.zip extracted to `./'.


'./'

Let's a mapping between the vocabulary in our pretained models and their corresponding embeddings.

In [ ]:
def load_embeddings():
  lines = open("./glove.6B.100d.txt", "r").readlines()

  w2e = {}
  for l in lines:
    s = l.split(" ")
    word = s[0]
    embedding = np.zeros( (1, len(s)-1))
    for k, x in enumerate(s[1:]):
      embedding[0,k] = float(x.strip())

    w2e[word] = embedding

  return w2e

w2e = load_embeddings()

We will be using the embedding layer from pytorch to build our network. To do so, we first need to create a weight matrix for each token in our vocabulary. That is, the weight matrix in row i should correspond to the word i in the vocabulary.

In [ ]:
def create_weight_matrix(w2e, token2idx, idx2token):
  vocab_length = len(list(token2idx.keys()))
  embed_dim = 100
  weight_matrix = np.zeros((vocab_length, embed_dim))

  for i in range(len(list(token2idx.keys()))):
    try:
        w = idx2token[i].lower()
        weight_matrix[i] = w2e[w]
    except KeyError:
        weight_matrix[i] = np.random.normal(scale=0.6, size=(embed_dim, ))

  return weight_matrix

Next, let's make a function that created the embedding layer and loads a weight matrix of pre-trained embeddings.

In [ ]:
def create_emb_layer(weight_matrix, non_trainable=False):
    num_embeddings = weight_matrix.shape[0]
    embedding_dim = weight_matrix.shape[1]
    weights = torch.from_numpy(weight_matrix)
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, embedding_dim

Next, let's build the network. Make sure you include a layer that is able to capture sequential dependency.

In [ ]:
#Sample solution
class Net(nn.Module):
    def __init__(self, weight_matrix, lstm_hidden_dim, num_of_tags):
        super(Net, self).__init__()

        #maps each token to an embedding_dim vector
        self.embedding, embedding_dim = create_emb_layer(weight_matrix)

        #the LSTM takens embedded sentence
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, batch_first=True)

        #fc layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, num_of_tags)



    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim

        #run the LSTM along the sentences of length batch_max_len
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim

        #reshape the Variable so that each row contains one token
        s = s.reshape(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim

        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags

        return F.log_softmax(s, dim=1)   # dim: batch_size*batch_max_len x num_tags

Create the loss function used for training our network

In [ ]:
# Sample solution
def loss_fn(outputs, labels):
    #reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)

    #the number of tokens is the sum of elements in mask
    num_tokens = labels.shape[0]

    #pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels]

    #cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs)/ num_tokens

In [ ]:
train_labels, val_labels, test_labels = [], [], []
for x in train_tags:
  train_labels.append( np.argmax(x, axis=1) )
train_labels = np.asarray(train_labels).squeeze()

for x in val_tags:
  val_labels.append( np.argmax(x, axis=1) )
val_labels = np.asarray(val_labels).squeeze()

for x in test_tags:
  test_labels.append( np.argmax(x, axis=1) )
test_labels = np.asarray(test_labels).squeeze()

print(train_labels.shape)
print(val_labels.shape)
print(test_labels.shape)

(32372, 104)
(10791, 104)
(4796, 104)


In [ ]:
# Create training routine

weight_matrix = create_weight_matrix(w2e, token2idx, idx2token)
network = Net( weight_matrix, lstm_hidden_dim=128, num_of_tags=len(list(idx2tag.keys())) )

optimizer = optim.Adam( network.parameters(), lr=1e-3 )

batch_size=128
indices = np.arange(train_labels.shape[0])
epochs = 2000

losses = []
for e in range(epochs):
  np.random.shuffle(indices)
  selected_indices = indices[:batch_size]
  selected_x = train_tokens[selected_indices]
  selected_y = train_labels[selected_indices]

  selected_x = torch.from_numpy(selected_x)
  selected_y = torch.from_numpy(selected_y)

  probs = network.forward(selected_x)
  loss = loss_fn(probs, selected_y)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  losses.append(loss.detach())
  if e % 100 == 0:
    print("Epoch: %d - %.6f" %(e, np.mean(losses)))
    losses = []

Epoch: 0 - 2.833374
Epoch: 100 - 0.363141
Epoch: 200 - 0.136530
Epoch: 300 - 0.095609
Epoch: 400 - 0.071030
Epoch: 500 - 0.051899
Epoch: 600 - 0.042326
Epoch: 700 - 0.035530
Epoch: 800 - 0.031702
Epoch: 900 - 0.030147
Epoch: 1000 - 0.027719
Epoch: 1100 - 0.026013
Epoch: 1200 - 0.024754
Epoch: 1300 - 0.023969
Epoch: 1400 - 0.022533
Epoch: 1500 - 0.021464
Epoch: 1600 - 0.020747
Epoch: 1700 - 0.020747
Epoch: 1800 - 0.020444
Epoch: 1900 - 0.019998


Now that we have our model trained. Let's evaluate using the test data in terms of precision, recall, and f1.

In [ ]:
#Evaluate precision-recall-f1
from sklearn.metrics import precision_recall_fscore_support

batch_size = 128
k = 0

preds = None
while k < test_tokens.shape[0]:
  x = test_tokens[k:k+batch_size] if k+batch_size < test_tokens.shape[0] else test_tokens[k:]
  y = test_labels[k:k+batch_size] if k+batch_size < test_labels.shape[0] else test_labels[k:]

  x = torch.from_numpy(x)
  y = torch.from_numpy(y)

  probs = network.forward(x).detach().numpy()
  yhat = np.argmax(probs, axis=1)

  preds = yhat if preds is None else np.hstack( (preds, yhat) )

  k += batch_size

labels = [ idx2tag[i] for i in range(len(list(idx2tag.keys()))) ]
test_labels = test_labels.reshape((-1,))

p, r, f, s = precision_recall_fscore_support( test_labels, preds)

for i in range(len(labels)):
  print("Label: %s - Precision: %.4f - Recall: %.4f - f1: %.4f - Support: %.4f" %(labels[i], p[i], r[i], f[i], s[i]) )

Label: B-nat - Precision: 1.0000 - Recall: 0.0588 - f1: 0.1111 - Support: 17.0000
Label: I-geo - Precision: 0.7683 - Recall: 0.7841 - f1: 0.7762 - Support: 681.0000
Label: I-gpe - Precision: 0.0000 - Recall: 0.0000 - f1: 0.0000 - Support: 22.0000
Label: I-nat - Precision: 0.0000 - Recall: 0.0000 - f1: 0.0000 - Support: 7.0000
Label: I-art - Precision: 0.0000 - Recall: 0.0000 - f1: 0.0000 - Support: 28.0000
Label: B-tim - Precision: 0.8795 - Recall: 0.8380 - f1: 0.8582 - Support: 1994.0000
Label: I-per - Precision: 0.8506 - Recall: 0.8799 - f1: 0.8650 - Support: 1715.0000
Label: B-art - Precision: 0.0000 - Recall: 0.0000 - f1: 0.0000 - Support: 28.0000
Label: I-eve - Precision: 0.0000 - Recall: 0.0000 - f1: 0.0000 - Support: 23.0000
Label: B-gpe - Precision: 0.9380 - Recall: 0.9300 - f1: 0.9340 - Support: 1628.0000
Label: O - Precision: 0.9976 - Recall: 0.9982 - f1: 0.9979 - Support: 482811.0000
Label: B-geo - Precision: 0.8222 - Recall: 0.8821 - f1: 0.8511 - Support: 3690.0000
Label: I

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**sample response: Classes with small number of examples tend to perform worse than others. This is likely due to data imbalance in training data. Possible ways to address that is, oversample minority class, undersample majority class, or balance mini-batches during training. Another option could also be to include all small classes as a single group, if that level of granularity is not needed**